In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds


def load_data():
    try:
        print("Loading datasets...")
        products = pd.read_csv('products.csv')
        ratings = pd.read_csv('ratings.csv')

        if 'productId' not in products.columns or 'title' not in products.columns:
            raise ValueError("Products dataset must have 'productId' and 'title' columns.")
        if 'userId' not in ratings.columns or 'productId' not in ratings.columns or 'rating' not in ratings.columns:
            raise ValueError("Ratings dataset must have 'userId', 'productId', and 'rating' columns.")

        print("Datasets loaded successfully.")
        return products, ratings
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None


def content_based_recommender(products, title, top_n=10):
    if 'description' not in products.columns:
        print("Error: 'description' column missing.")
        return []

    try:
        tfidf = TfidfVectorizer(stop_words='english')
        products['description'] = products['description'].fillna('')
        tfidf_matrix = tfidf.fit_transform(products['description'])

        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        if title not in products['title'].values:
            print(f"Error: Product '{title}' not found.")
            return []

        idx = products[products['title'] == title].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        top_products = [products['title'].iloc[i[0]] for i in sim_scores[1:top_n+1]]
        return top_products
    except Exception as e:
        print(f"Error in content-based recommender: {e}")
        return []


def collaborative_filtering(ratings):
    try:
        user_item_matrix = ratings.pivot(index='userId', columns='productId', values='rating').fillna(0)
        user_ratings_mean = np.mean(user_item_matrix, axis=1)
        matrix_norm = user_item_matrix - user_ratings_mean.values.reshape(-1, 1)

        U, sigma, Vt = svds(matrix_norm, k=50)
        sigma = np.diag(sigma)

        predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.values.reshape(-1, 1)
        predictions = pd.DataFrame(predicted_ratings, columns=user_item_matrix.columns)
        return predictions
    except Exception as e:
        print(f"Error in collaborative filtering: {e}")
        return pd.DataFrame()


def hybrid_recommender(user_id, title, products, ratings, top_n=10):
    try:
        content_recs = content_based_recommender(products, title, top_n=top_n)
        if not content_recs:
            print("No content-based recommendations available.")
            return []

        predictions = collaborative_filtering(ratings)
        if user_id not in predictions.index:
            print(f"Error: User ID '{user_id}' not found.")
            return []

        user_predictions = predictions.loc[user_id].sort_values(ascending=False)
        hybrid_recs = list(set(content_recs + user_predictions.index[:top_n].tolist()))
        return hybrid_recs[:top_n]
    except Exception as e:
        print(f"Error in hybrid recommender: {e}")
        return []


if __name__ == "__main__":
    products, ratings = load_data()
    if products is None or ratings is None:
        exit()


    user_id = 1
    product_title = "Example Product"
    try:
        recommendations = hybrid_recommender(user_id, product_title, products, ratings, top_n=10)
        print("Recommendations:", recommendations)
    except Exception as e:
        print(f"Unexpected error: {e}")

